In [ ]:
선형회귀(LinearRegression)

한개이상의 독립변수와 종속변수와의 선형적인 상관관계를 모델링하는 회귀 분석 기법

독립변수 개수에따라 하나면 단순 선형회귀, 두개 이상이면 다중 선형회귀라고함

일반적으로 최소 제곱법(least square method) 을 사용회 선형 회귀 모델을 만듦

sklearn, statsmodels 등의 패키지를 이용하여 회귀분석을 수행한다.

In [2]:
# 선형 회귀 LinearRegression sklearn
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
datas = datasets.load_diabetes()
datas.target
df = pd.DataFrame(datas.data, columns = datas.feature_names)
y = pd.DataFrame(datas.target, columns = ['y'])
df = pd.concat([df, y], axis=1)
train, test = train_test_split(df, test_size=0.3, random_state=123)
cols = train.columns
from sklearn.linear_model import LinearRegression
# 독립변수 == 설명 변수
# 선형회귀 - 절편 있는 경우
lr = LinearRegression()
# 절편(intercept) 없는 경우
# lr = LinearRegression(fit_intercept=False)
lr.fit(train.iloc[:, :-1], train.y)

pred = lr.predict(test.iloc[:, :-1])
# print(pred)

# 회귀 계수 구할떄
lr.coef_

# 영향력이 큰 또는 예측값이 큰 변수 구할떄 = 회귀계수 가장 큰값과 해당 변수명 구하기
print(lr.coef_[lr.coef_.argmax()])
print(cols[lr.coef_.argmax()])
# 절편 구할떄
lr.intercept_

# [1,1,1,1,1,1,1,1,1,?] 이고 y = 100 일떄 ? 값
arr = [1,1,1,1,1,1,1,1,1, '?']
arr = arr[:-1]
ans = (100 - lr.intercept_ - np.dot(arr, lr.coef_[:-1].reshape(-1, 1)))/lr.coef_[-1]
print(ans)

# 결정계수
# 선형회귀분석에서 종속변수에 대한 설명변수들의 설명력을 알고자 할때 사용
# 총변동중 회귀선에 의한 변동이 차지하는 비율 
# r2 = 0.4 == 약 40% 설명
from sklearn.metrics import r2_score

# 결정 계수
print(r2_score(test.y, pred))

684.0489521950243
s5
[-13.66400969]
0.5078285584893742


In [3]:
# 선형 회귀 LinearRegression - statsmodel.formula.api 사용
from statsmodels.formula.api import ols
s = 'y ~ '
for i, c in enumerate(cols[:-1]):
  if i==0:
    s = s + c
    continue
  s = s +'+'+ c 
# 아래 summary에서 P>|t| 를 통해 유의미한 개수 확인 가능 => 0.05 보다 작은 경우 유의미함
ols(s, df).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.518
Model:                            OLS   Adj. R-squared:                  0.507
Method:                 Least Squares   F-statistic:                     46.27
Date:                Thu, 17 Sep 2020   Prob (F-statistic):           3.83e-62
Time:                        21:47:16   Log-Likelihood:                -2386.0
No. Observations:                 442   AIC:                             4794.
Df Residuals:                     431   BIC:                             4839.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    152.1335      2.576     59.061      0.000     147.071     157.196
age          -10.0122     59.749     -0.168      0.867    -127.448     107.424
sex         -239.8191     61.222     -3.917      0.000    -360.151    -119.488
bmi          519.8398     66.534      7.813      0.000     389.069     650.610
bp           324.3904     65.422      4.958      0.000     195.805     452.976
s1          -792.1842    416.684     -1.901      0.058   -1611.169      26.801
s2           476.7458    339.035      1.406      0.160    -189.621    1143.113
s3           101.0446    212.533      0.475      0.635    -316.685     518.774
s4           177.0642    161.476      1.097      0.273    -140.313     494.442
s5           751.2793    171.902      4.370      0.000     413.409    1089.150
s6            67.6254     65.984      1.025      0.306     -62.065     197.316
==============================================================================
Omnibus:                        1.506   Durbin-Watson:                   2.029
Prob(Omnibus):                  0.471   Jarque-Bera (JB):                1.404
Skew:                           0.017   Prob(JB):                        0.496
Kurtosis:                       2.726   Cond. No.                         227.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [4]:
# 선형 회귀 LinearRegression - statsmodel.api 사용
from statsmodels.api import OLS, add_constant
# const 칼럼 추가
# add_constant 안하면 r2 score 구할떄에 uncentered 되어 조금 다르게 나옴
train_data = add_constant(train)
model = OLS(train_data.y, train_data.drop(columns='y')).fit()
# 유의미한 변수 확인
p_cols = model.pvalues.index[model.pvalues <=0.05]
print(p_cols)
# 유의미한 변수로 재학습
train_data_x = train_data[list(p_cols)[1:] + ['y']]
train_data_x = add_constant(train_data_x)
train_data_x

model = OLS(train_data_x.y, train_data_x.drop(columns='y')).fit()
test_data_x = test[list(p_cols[1:]) + ['y']]
test_data_x = add_constant(test_data_x)
pred = model.predict(test_data_x.drop(columns='y'))
# 모델 살펴보기
print(model.summary())
# 모델 coefficient 추출
print(model.params)
# 모델 p_valeus 추출
print(model.pvalues)
# 모델 t_values
print(model.tvalues)
# 결정계수
print(model.rsquared)
# 수정된 결정 계수
print(model.rsquared_adj)

Index(['const', 'sex', 'bmi', 'bp', 's5'], dtype='object')
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.478
Model:                            OLS   Adj. R-squared:                  0.471
Method:                 Least Squares   F-statistic:                     69.53
Date:                Thu, 17 Sep 2020   Prob (F-statistic):           9.62e-42
Time:                        21:47:19   Log-Likelihood:                -1680.0
No. Observations:                 309   AIC:                             3370.
Df Residuals:                     304   BIC:                             3389.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------

C:\Users\carl\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
